In [ ]:
import openpyxl
import requests
import time
from bs4 import BeautifulSoup
from google.colab import files
import os
import pandas as pd

In [ ]:
# Load the Excel file
year = '2019' # change accordingly
file_name = year+' Merged Lists.xlsx'
sheet_name = year+' Merged'

start = 201 # starting row/company
end = 300 #ending row/company

workbook = openpyxl.load_workbook(file_name) # specify the file name
sheet = workbook[sheet_name] # specify the sheet, otherwise use workbook.active to refer to the 1st sheet in the file

In [ ]:
companies_data = sheet.iter_rows(min_row=start, max_row=end, values_only=True) # specify the starting row and finishing row [3-505] (can download all in 1 go, but it will be so long so maybe better to break it down)

In [ ]:
identifiers = []
company_names = []
links = []

# Get the identifiers, company names, and links from the Excel file

for row in companies_data:
  identifiers.append(row[0])
  company_names.append(row[1])
  links.append(row[4])

In [ ]:
len(identifiers), identifiers[-2:], company_names[-2:], links[-2:] # checking only

(100,
 ['L.N', 'LOW.N'],
 ['Loews Corp', "Lowe's Companies Inc"],
 ['#N/A',
  'https://1g0r7s45brd833po5f1d5yyb-wpengine.netdna-ssl.com/wp-content/uploads/2019/07/Lowes_2018CRReport_071919.pdf'])

In [ ]:
# Create a directory to save the downloaded files
folder = year+'_downloaded_files_'+str(start)+'_'+str(end)

if not os.path.exists(folder):
    os.makedirs(folder) # create new folder to store all found documents

# Create a DataFrame to store the download results
download_results = pd.DataFrame(columns=['Identifier', 'Company Name', 'File Name', 'Downloaded'])

In [ ]:
# Set the maximum download time (in seconds)
max_download_time = 5 # recommend 10 s

In [ ]:
for identifier, company_name, link in zip(identifiers, company_names, links):

    file_status = ''
    download_status = False

    if link == '#N/A':  # skip companies that still don't have a link
        print('Link is N/A')
        file_status = 'N/A'

    else:
        start_time = time.time()

        try:
            # Make the HTTP request and get the response
            response = requests.get(link, stream=True, timeout=max_download_time)

            # Check if the response is successful
            response.raise_for_status()

            # Generate the file name
            file_name = f"{identifier}_{year}.pdf"
            file_path = os.path.join(folder, file_name)

            # Save the file to disk
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

            print(f'Downloaded: {file_name}')
            file_status = file_name
            download_status = True

        except requests.exceptions.Timeout:
            print(f"Download of {link} took longer than {max_download_time} seconds. Skipping this file.")
            file_status = 'Timed Out, link: '+link

        except Exception as e:
            print(f'Error downloading {link}: {e}')
            file_status = 'Invalid'

    new_row = pd.DataFrame({'Identifier': [identifier], 'Company Name': [company_name], 'File Name': [file_status], 'Downloaded': [download_status]})
    download_results = pd.concat([download_results, new_row], ignore_index=True)


Error downloading http://www.firstsolar.com/en/Global/Sustainability-Documents/Sustainability-Report---2020?dl=1: 500 Server Error: Internal Server Error for url: https://www.firstsolar.com/sitecore/service/nolayout.aspx?item=%2fglobal%2fsustainability-documents%2fsustainability-report---2020&layout=%7b00000000-0000-0000-0000-000000000000%7d&device=Default
Downloaded: FE.N_2019.pdf
Link is N/A
Downloaded: FMC.N_2019.pdf
Error downloading https://s23.q4cdn.com/799033206/files/doc_downloads/esg/2020/06/Ford-sr20.pdf: 403 Client Error: Forbidden for url: https://s23.q4cdn.com/799033206/files/doc_downloads/esg/2020/06/Ford-sr20.pdf
Downloaded: FTNT.OQ_2019.pdf
Error downloading https://www.fortive.com/sites/default/files/inline-files/55819-199_Fortive_SustainabilityReport_AW3_ONLINE.pdf: 404 Client Error: Not Found for url: https://www.fortive.com/sites/default/files/inline-files/55819-199_Fortive_SustainabilityReport_AW3_ONLINE.pdf
Link is N/A
Link is N/A
Link is N/A
Downloaded: FCX.N_201

In [ ]:
# Save the download results to an Excel file
excel_file = year+'_download_results_'+str(start)+'_'+str(end)+'.xlsx'
download_results.to_excel(excel_file, index=False)
files.download(f"/content/{excel_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download the folder with all files to local
zip_file = f"{folder}.zip"
os.system(f"zip -r {zip_file} /content/{folder}")

files.download(f"/content/{zip_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>